In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

In [12]:
baseline_dir = Path('Fusion_trials_baseline')
fusion_dir = Path('Fusion_trials')

In [13]:
print(baseline_dir.is_dir())
print(fusion_dir.is_dir())

True
True


In [14]:
co2_caps = [100.0, 2.5, 2.0, 1.5, 0.5, 0.25, 0.1]

In [15]:
load_df = pd.read_csv(fusion_dir.joinpath('Load_data.csv'))
load_df.head()

,Voll,Demand_Segment,Cost_of_Demand_Curtailment_per_MW,Max_Demand_Curtailment,Rep_Periods,Timesteps_per_Rep_Period,Sub_Weights,Time_Index,Load_MW_z1
0,50000.0,1.0,1.0,1.0,1.0,168.0,1.0,1,15068.005
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,14398.244
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,13772.792
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,13106.453
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,12517.167


In [16]:
total_load = load_df.loc[:, 'Load_MW_z1'].sum()

In [17]:
cases = ['Results', 'Results_1', 'Results_2', 'Results_3', 'Results_7', 'Results_4', 'Results_5', 'Results_6']

In [18]:
emiss_levels = []
for case in cases:
    temp = pd.read_csv(fusion_dir.joinpath(case, 'emissions.csv'))
    emiss_levels.append(temp.loc[1, 'Total'])
emiss_intensity = np.array(emiss_levels) / total_load * 1e6
print(emiss_intensity) 

[178.60595038  20.44560521  16.35648417  12.26736313   8.17824208
   4.08912104   2.04456052   0.81782421]


In [19]:
data = pd.DataFrame(index=cases)
data.loc[:, 'CO2_intensity'] = emiss_intensity
data

,CO2_intensity
Results,178.605950
Results_1,20.445605
Results_2,16.356484
Results_3,12.267363
Results_7,8.178242
Results_4,4.089121
Results_5,2.044561
Results_6,0.817824


In [20]:
cap_df = pd.DataFrame(columns=cases)
# cap_df.set_index(temp.index, inplace=True)
for case in cases:
    temp = pd.read_csv(fusion_dir.joinpath(case, 'capacity.csv'), index_col='Resource')
    cap_df.loc[:,case] = temp.loc[:,'EndCap'].values
cap_index = temp.index
cap_df.set_index(cap_index, inplace=True)

C:\Users\Ruaridh\AppData\Local\Temp\ipykernel_20864\1633185872.py:5: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  cap_df.loc[:,case] = temp.loc[:,'EndCap'].values
C:\Users\Ruaridh\AppData\Local\Temp\ipykernel_20864\1633185872.py:5: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  cap_df.loc[:,case] = temp.loc[:,'EndCap'].values
C:\Users\Ruaridh\AppData\Local\Temp\ipykernel_20864\1633185872.py:5: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old b

In [21]:
cap_df

,Results,Results_1,Results_2,Results_3,Results_7,Results_4,Results_5,Results_6
Resource,,,,,,,,
natural_gas_ccs,1.005113e-11,7.457913e+03,8.497873e+03,9.711962e+03,9.798707e+03,7572.990635,5328.026444,3.388865e+03
solar_pv,7.365267e+03,1.523408e+04,1.517293e+04,1.620703e+04,1.864776e+04,19202.164957,17674.546828,1.886180e+04
onshore_wind,1.487593e+04,2.606001e+04,2.622567e+04,2.691155e+04,3.133581e+04,29768.441446,24945.587388,1.936850e+04
battery,1.817556e+03,5.661519e+03,5.630782e+03,5.343378e+03,5.824739e+03,6102.705512,6592.075865,7.285672e+03
offshore_wind,2.090885e-12,7.384435e-13,5.234268e-12,1.765383e-10,5.535492e-10,328.063377,204.622875,4.653096e+02
natural_gas,1.965687e+04,5.317210e+03,4.284995e+03,3.103720e+03,1.523404e+03,709.971364,447.867718,1.308071e-08
nuclear,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00
fusion,7.257964e-12,4.528532e-12,3.262468e-11,1.338724e-09,4.783771e-09,4317.806670,9132.717079,1.308873e+04
Total,4.371562e+04,5.973073e+04,5.981224e+04,6.127764e+04,6.713041e+04,68002.143960,64325.444197,6.245887e+04


In [22]:
gen_df = pd.DataFrame(columns=cases)
for case in cases:
    temp = pd.read_csv(fusion_dir.joinpath(case, 'power.csv'), index_col='Resource')
    gen_df.loc[:,case] = temp.loc['AnnualSum',:].values
gen_df.set_index(cap_index, inplace=True)
gen_df

C:\Users\Ruaridh\AppData\Local\Temp\ipykernel_20864\3700687827.py:4: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  gen_df.loc[:,case] = temp.loc['AnnualSum',:].values
C:\Users\Ruaridh\AppData\Local\Temp\ipykernel_20864\3700687827.py:4: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  gen_df.loc[:,case] = temp.loc['AnnualSum',:].values
C:\Users\Ruaridh\AppData\Local\Temp\ipykernel_20864\3700687827.py:4: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the

,Results,Results_1,Results_2,Results_3,Results_7,Results_4,Results_5,Results_6
Resource,,,,,,,,
natural_gas_ccs,3.548814e-08,2.899754e+07,3.048627e+07,3.053543e+07,2.388769e+07,1.256850e+07,6.352598e+06,2.629823e+06
solar_pv,9.145295e+06,1.699791e+07,1.674791e+07,1.768846e+07,1.935082e+07,2.010807e+07,1.975752e+07,2.229255e+07
onshore_wind,4.856557e+07,7.339867e+07,7.380588e+07,7.433332e+07,8.048964e+07,7.837792e+07,7.186270e+07,6.046171e+07
battery,7.606510e+05,6.925472e+06,6.912950e+06,7.082449e+06,9.500991e+06,9.364630e+06,8.202901e+06,6.740708e+06
offshore_wind,5.830291e-09,2.045270e-09,1.440647e-08,4.757508e-07,1.473377e-06,9.583563e+05,6.517345e+05,1.639028e+06
natural_gas,6.470284e+07,4.138345e+06,2.490137e+06,1.003738e+06,2.717089e+05,6.552629e+04,2.510898e+04,7.638956e-07
nuclear,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
fusion,3.349979e-08,2.157716e-08,1.536582e-07,6.444319e-06,2.160400e-05,1.189674e+07,2.511462e+07,3.647581e+07
Total,1.231744e+08,1.304579e+08,1.304431e+08,1.306434e+08,1.335008e+08,1.333397e+08,1.319672e+08,1.302396e+08


In [23]:
CF_df = gen_df / cap_df / 8760
CF_df

,Results,Results_1,Results_2,Results_3,Results_7,Results_4,Results_5,Results_6
Resource,,,,,,,,
natural_gas_ccs,0.403055,0.443854,0.409534,0.358916,0.278292,0.189458,0.136107,0.088587
solar_pv,0.141744,0.127372,0.126005,0.124590,0.118459,0.119541,0.127609,0.134919
onshore_wind,0.372684,0.321521,0.321263,0.315312,0.293221,0.300562,0.328856,0.356353
battery,0.047774,0.139641,0.140149,0.151309,0.186204,0.175172,0.142050,0.105617
offshore_wind,0.318314,0.316176,0.314194,0.307636,0.303846,0.333476,0.363590,0.402106
natural_gas,0.375755,0.088846,0.066339,0.036918,0.020360,0.010536,0.006400,0.006667
nuclear,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fusion,0.526894,0.543917,0.537657,0.549518,0.515537,0.314529,0.313923,0.318129
Total,0.321647,0.249327,0.248959,0.243378,0.227018,0.223838,0.234196,0.238037


In [28]:
temp

,Costs,Total,Zone1
0,cTotal,6.118822e+09,6.118821698653816e9
1,cFix,4.315881e+09,4.31588056808614e9
2,cVar,1.775446e+09,1.7754460051381772e9
3,cNSE,1.722070e-07,1.722069877300218e-7
4,cStart,2.749513e+07,2.749512542949942e7
5,cUnmetRsv,0.000000e+00,-
6,cNetworkExp,0.000000e+00,-


In [33]:
costs_df = pd.DataFrame(columns=cases)
for case in cases:
    temp = pd.read_csv(fusion_dir.joinpath(case, 'costs.csv'))
    costs_df.loc[0,case] = temp.loc[0,'Total']
# costs_df.set_index(cap_index, inplace=True)
costs_df

,Results,Results_1,Results_2,Results_3,Results_7,Results_4,Results_5,Results_6
0,6118821698.653811,7731011975.3117,7878220593.718235,8079431002.231201,8456462570.524051,9391310918.824881,10160280269.664368,10971752818.992085


In [35]:
baseline_costs = pd.DataFrame(columns=cases)
for case in cases:
    temp = pd.read_csv(baseline_dir.joinpath(case, 'costs.csv'))
    baseline_costs.loc[0,case] = temp.loc[0,'Total']
# costs_df.set_index(cap_index, inplace=True)
baseline_costs

,Results,Results_1,Results_2,Results_3,Results_7,Results_4,Results_5,Results_6
0,6118821698.653817,7731011975.314357,7878220593.718048,8079431002.231242,8456462570.52178,9520693584.700771,10807679345.818144,12624322073.438295
